In [5]:
import pandas as pd
quakes = pd.read_csv('../data_input/5_USGS_quakes/nz.csv')

In [7]:
print(quakes.shape)
quakes.head()

(258, 8)


,id,time,location,latitude,longitude,depth,magnitude,url
0,usp000jxzs,2013-01-06 16:54:47.890,east of the North Island of New Zealand,-35.035,-179.575,65.8,4.2,https://earthquake.usgs.gov/earthquakes/eventp...
1,usp000jy10,2013-01-07 13:37:39.200,south of the Kermadec Islands,-34.807,179.905,35.0,4.3,https://earthquake.usgs.gov/earthquakes/eventp...
2,usp000jy17,2013-01-07 16:22:23.370,off the east coast of the North Island of New ...,-36.767,177.617,182.1,4.5,https://earthquake.usgs.gov/earthquakes/eventp...
3,usp000jy26,2013-01-08 11:43:29.240,east of the North Island of New Zealand,-35.141,-179.481,35.0,4.8,https://earthquake.usgs.gov/earthquakes/eventp...
4,usp000jy2w,2013-01-08 21:26:55.640,south of the Kermadec Islands,-34.601,179.180,10.0,5.7,https://earthquake.usgs.gov/earthquakes/eventp...


In [13]:
## Query generator

# Lower 48 USA
lon = (-125.0011,-66.9326)
lat = (24.9493, 49.5904)
start = '2008-01-01'
end = '2019-01-01'
name = 'lower48.csv'

minmag = 4
maxmag = 9.9

print(f'getcsv {name} -b {lon[0]} {lon[1]} {lat[0]} {lat[1]} -s {start} -e {end} -f csv -x -m {minmag} {maxmag}')

getcsv lower48.csv -b -125.0011 -66.9326 24.9493 49.5904 -s 2008-01-01 -e 2019-01-01 -f csv -x -m 4 9.9
